# Effect of XGBoost on Stock Market Prediction

In this notebook we will have a look into XGBoost for stock market prediction

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Gathering data
# TODO : Create function to get data from yfinance to be used in future projects
ticker =  "AMZN"
today = date.today()

amzn = yf.download(ticker, start = "2020-01-01", end = date.today() )

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# Verifying there are no null or na values
print(f"NA Sum: \n{amzn.isna().sum()}\n")
print(f"Null Sum: \n{amzn.isnull().sum()}")

NA Sum: 
Price   Ticker
Close   AMZN      0
High    AMZN      0
Low     AMZN      0
Open    AMZN      0
Volume  AMZN      0
dtype: int64

Null Sum: 
Price   Ticker
Close   AMZN      0
High    AMZN      0
Low     AMZN      0
Open    AMZN      0
Volume  AMZN      0
dtype: int64
